In [141]:
sc.version

u'1.4.1'

In [178]:
path_to_data = "/Users/kev/largeDataTeam/data_files/*/*"
path_to_train = "/Users/kev/largeDataTeam/data/train.csv"

data = sc.wholeTextFiles(path_to_data)

from bs4 import BeautifulSoup
def parser(x):
    return BeautifulSoup(x, 'html.parser')

In [179]:
import graphlab as gl
train = gl.SFrame(path_to_train)
t = list(train['file'])

PROGRESS: Finished parsing file /Users/kev/largeDataTeam/data/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.64723 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/kev/largeDataTeam/data/train.csv
PROGRESS: Parsing completed. Parsed 101107 lines in 0.275713 secs.


In [180]:
mapped_data = data.map(lambda x: [x[0].split("/")[-1], parser(x[1]).get_text(), "train" if x[0].split("/")[-1] in t else "test" ])

mapped_train_data = mapped_data.filter(lambda x: x[2]=="train")

In [181]:
tokenized_html = mapped_train_data.map(lambda x: x[1]).flatMap(lambda a: a.split(' ')).map(lambda word: (word, 1))

In [ ]:
from operator import add

reduced_html = tokenized_html.reduceByKey(add)

In [ ]:
print new_reduced_html.take(1)